new type test - a custom type with its own validators is added to the schema

In [1]:
import copy
from collections import namedtuple

import jsonschema
from jsonschema import Draft7Validator, ValidationError

In [2]:
Dimensions = namedtuple("Dimensions", "length, width, height")

In [3]:
def maxLengthChecker(validator, max_length, instance, schema):
    if not validator.is_type(instance, "dimensions"):
        return
    if max(instance) > max_length:
        yield ValidationError(f"longest dimension is greater than the maximum length of {max_length}")
        
def maxVolumeChecker(validator, max_volume, instance, schema):
    if not validator.is_type(instance, "dimensions"):
        return
    volume = instance.length * instance.width * instance.height
    if volume > max_volume:
        yield ValidationError(f"total volume {volume!r} is greater than the maximum of {max_volume}")

In [4]:
type_checker = Draft7Validator.TYPE_CHECKER.redefine(
    "dimensions", lambda _, instance: isinstance(instance, Dimensions)
)
format_checker = Draft7Validator.FORMAT_CHECKER
all_validators = dict(Draft7Validator.VALIDATORS)
all_validators['maximumLength'] = maxLengthChecker
all_validators['maximumVolume'] = maxVolumeChecker
meta_schema = copy.deepcopy(Draft7Validator.META_SCHEMA)
meta_schema['definitions']['simpleTypes']['enum'].append("dimensions")
meta_schema['properties']['maximumLength'] = {'type': 'number', 'minimum': 0}
meta_schema['properties']['maximumVolume'] = {'type': 'number', 'minimum': 0}

MyValidator = jsonschema.validators.create(
    meta_schema=meta_schema,
    validators=all_validators,
    type_checker=type_checker,
    format_checker=Draft7Validator.FORMAT_CHECKER,
    id_of=Draft7Validator.ID_OF
)

In [5]:
schema = {
    "type": "dimensions",
    "maximumLength": 10,
    "maximumVolume": 250
}
validator = MyValidator(schema)

In [6]:
validator.validate(Dimensions(8, 5, 6))

In [7]:
list(validator.iter_errors(Dimensions(11, 0.5, 0.3)))

[<ValidationError: 'longest dimension is greater than the maximum length of 10'>]

In [8]:
list(validator.iter_errors(Dimensions(8, 5, 7)))

[<ValidationError: 'total volume 280 is greater than the maximum of 250'>]